In [2]:
import torch, time, copy
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch import nn, optim
from TransformDataset import TransformDataset

In [3]:
model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
model = model.float()

In [4]:
# Number of classes in the dataset
num_classes = 10

# Batch size for training (change depending on how much memory you have)
batch_size = 1024

# Number of epochs to train for
num_epochs = 15

input_size = 224

device = 'mps'

In [10]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [11]:
model.classifier[6] = nn.Linear(model.classifier[6].in_features,num_classes)

In [12]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

cifar_train = datasets.CIFAR10('data', True, download=True)
cifar_train, cifar_val = torch.utils.data.random_split(cifar_train, [40000, 10000])
cifar_test = datasets.CIFAR10('data', False, data_transforms['val'], download=True)

# Create training and validation datasets
image_datasets = {
    'train': TransformDataset(cifar_train, data_transforms['train']),
    'val': TransformDataset(cifar_val, data_transforms['val']),
    'test': cifar_test
}

# Create training and validation dataloaders
dataloaders_dict = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4)
    for x in ['train', 'val', 'test']
}

Initializing Datasets and Dataloaders...
Files already downloaded and verified
Files already downloaded and verified


In [13]:
model = model.to(device)
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model.parameters()
print("Params to learn:")
for name,param in model.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 features.0.weight
	 features.0.bias
	 features.3.weight
	 features.3.bias
	 features.6.weight
	 features.6.bias
	 features.8.weight
	 features.8.bias
	 features.10.weight
	 features.10.bias
	 classifier.1.weight
	 classifier.1.bias
	 classifier.4.weight
	 classifier.4.bias
	 classifier.6.weight
	 classifier.6.bias


In [14]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model, hist = train_model(model, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/14
----------
train Loss: 1.5006 Acc: 0.4605
val Loss: 0.7972 Acc: 0.7241

Epoch 1/14
----------
train Loss: 1.0799 Acc: 0.6162
val Loss: 0.6558 Acc: 0.7771

Epoch 2/14
----------
train Loss: 1.0059 Acc: 0.6432
val Loss: 0.6066 Acc: 0.7895

Epoch 3/14
----------
train Loss: 0.9569 Acc: 0.6618
val Loss: 0.5745 Acc: 0.8005

Epoch 4/14
----------
train Loss: 0.9289 Acc: 0.6717
val Loss: 0.5376 Acc: 0.8152

Epoch 5/14
----------
train Loss: 0.9065 Acc: 0.6794
val Loss: 0.5053 Acc: 0.8243

Epoch 6/14
----------
train Loss: 0.8873 Acc: 0.6865
val Loss: 0.4916 Acc: 0.8301

Epoch 7/14
----------
train Loss: 0.8836 Acc: 0.6873
val Loss: 0.4913 Acc: 0.8297

Epoch 8/14
----------
train Loss: 0.8564 Acc: 0.6974
val Loss: 0.4729 Acc: 0.8384

Epoch 9/14
----------
train Loss: 0.8460 Acc: 0.6995
val Loss: 0.4574 Acc: 0.8399

Epoch 10/14
----------
train Loss: 0.8397 Acc: 0.7029
val Loss: 0.4446 Acc: 0.8420

Epoch 11/14
----------
train Loss: 0.8276 Acc: 0.7075
val Loss: 0.4375 Acc: 0.8481

Ep

In [17]:
correct = 0
total = len(dataloaders_dict['test'].dataset)
model.eval()

# Iterate over data.
for inputs, labels in dataloaders_dict['test']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # track history if only in train
    with torch.no_grad():
        # Get model outputs and calculate loss
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

    # statistics
    correct += torch.sum(preds == labels.data)

print(f'test Acc: {correct.float()/total}')

test Acc: 0.8592000603675842
